## Explaining MNIST Image Classifiers with CXPlain

First, we load 500 sample images from the MNIST dataset which we will use to attempt to recognise the digit type
that a given image shows.

In [1]:
import sys
sys.path.append("./cxplain/")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from cxplain.util.test_util import TestUtil

num_subsamples = 500
(x_train, y_train), (x_test, y_test) = TestUtil.get_mnist(flattened=False, num_subsamples=num_subsamples)

/home/tchiaburu/anaconda3/envs/tf2_8_gpu_cxplain/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Next, we fit a multilayer perceptron (MLP) that predicts the digit type that a given MNIST image shows.

In [2]:
from sklearn.neural_network import MLPClassifier

explained_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                                hidden_layer_sizes=(64, 32), random_state=1)
explained_model.fit(x_train.reshape((len(x_train), -1)), y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(64, 32), random_state=1,
              solver='lbfgs')

After fitting the multilayer perceptron, we wish to explain its decisions, i.e. what input features were most relevant
for a given model prediction. To do so, we train a causal explanation (CXPlain) model that can learn to explain any
machine-learning model using the same training data. In practice, we have to define:
- `model_builder`: The type of model we want to use as our CXPlain model. In this case we are using a neural explanation model using
a UNet structure. 
- `masking_operation`: The masking operaion used to remove a certain input feature from the set of available input features. In this case we are using zero masking, i.e. setting the value of that input feature to zero.
- `loss`: The loss function that we wish to use to measure the impact of removing a certain input feature from the set of available features. In most common use cases, this will be the mean squared error (MSE) for regression problems and the cross-entropy for classification problems.


In [3]:
from tensorflow.keras.losses import categorical_crossentropy
from cxplain import UNetModelBuilder, ZeroMasking, CXPlain

downsample_factors = (2, 2)
model_builder = UNetModelBuilder(downsample_factors, num_layers=2, num_units=8, activation="relu",
                                 p_dropout=0.0, verbose=0, batch_size=32, learning_rate=0.001)
masking_operation = ZeroMasking()
loss = categorical_crossentropy

Using this configuration, we now instantiate a CXPlain model and fit it to the same MNIST training data that we used to fit the MLP model that we wish to explain.
We additionally request that 5 bootstrap resampled versions of this model are trained in order to be able to compute confidence intervals for the estimated importance scores.
Only a single model is trained - and computation of confidence intervals is not possible - if you do not specify the number of models you wish to train.

In [4]:
explainer = CXPlain(explained_model, model_builder, masking_operation, loss, 
                    num_models=5, downsample_factors=downsample_factors, flatten_for_explained_model=True)
explainer.fit(x_train, y_train)

NotImplementedError: in user code:

    File "/mnt/KInsektDaten/teo/./cxplain/cxplain/backend/tensorflow_cxplain.py", line 163, in _fit_single  *
        masked_data = self.masking_operation.get_predictions_after_masking(self.explained_model, X, y,
    File "/mnt/KInsektDaten/teo/./cxplain/cxplain/backend/masking/zero_masking.py", line 39, in get_predictions_after_masking  *
        y_pred = MaskingUtil.get_prediction(explained_model, x, flatten=flatten)
    File "/mnt/KInsektDaten/teo/./cxplain/cxplain/backend/masking/masking_util.py", line 200, in get_prediction  *
        x = np.reshape(x, (len(x), -1))
    File "/home/tchiaburu/anaconda3/envs/tf2_8_gpu_cxplain/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 285, in reshape  **
        return _wrapfunc(a, 'reshape', newshape, order=order)
    File "/home/tchiaburu/anaconda3/envs/tf2_8_gpu_cxplain/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 56, in _wrapfunc
        return _wrapit(obj, method, *args, **

We can then use this fitted CXPlain model to explain the predictions of the explained model on the held-out test samples.
We additionally request confidence intervals at the 80% confidence level, i.e. the 10% and 90% quantiles of the observed output distribution of importance scores.
Note that the importance scores are normalised to sum to a value of 1 and each score therefore represents the relative importance of each respective input feature.

In [ ]:
attributions, confidence = explainer.explain(x_test, confidence_level=0.80)

We can now visualise the 2x2 pixel block attributions and their confidence for a specific sample from the test set using the `Plot` toolset available as part of CXPlain.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cxplain.visualisation.plot import Plot

plt.rcdefaults()

selected_index = 2
selected_sample = x_test[selected_index]
importances = attributions[selected_index]
importances_confidence = confidence[selected_index]

Plot.plot_attribution_2d(selected_sample,
                         importances, 
                         importances_confidence)

In [ ]:
selected_index = 3
selected_sample = x_test[selected_index]
importances = attributions[selected_index]
importances_confidence = confidence[selected_index]

Plot.plot_attribution_2d(selected_sample,
                         importances, 
                         importances_confidence)

